In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics
import anndata as an
import scanpy as sp
from scipy.stats import mannwhitneyu
from sklearn.metrics import roc_auc_score
tissues = ['bladder','brain','diaphragm','fat BAT','fat GAT','fat MAT','fat SCAT','heart','kidney','large intestine','limb muscle','liver','lung','marrow','pancreas','skin','spleen','thymus','tongue','trachea']

In [ ]:
def run_stat(X,C,T,y,ii):
        U , p = mannwhitneyu(C[ii,:],T[ii,:])
        auc = roc_auc_score(y,X[ii,:])-0.5
        return p,auc

def dge(d,col,control,treatment,sample=False,num=10):
        X = np.array(d.X.todense().transpose())
        genes = np.array(d.var.index)
        a = d.obs
        control = np.array(a[col]==control)
        treatment = np.array(a[col]==treatment)
        num_control = control.sum()
        num_treatment = treatment.sum()
        num_genes = len(genes)
        res = {}
        for key in ['log','auc','nzc','nzt']:
            res[key] = np.zeros((num_genes,))
        res['pval'] = np.ones((num_genes,))
        res['num_control'] = num_control
        res['num_treatment'] = num_treatment
        if num_control >= 1 and num_treatment >= 1:
            C = X[:,control].copy()
            T = X[:,treatment].copy()
            y = np.concatenate([np.zeros((C.shape[1],)),np.ones((T.shape[1],))])
            res['nzc'] = (C>0).sum(axis=1)
            res['nzt'] = (T>0).sum(axis=1)
            gene_filter_  = (res['nzc']>0) & (res['nzt']>0)
            X_ = np.concatenate([C,T],axis=1)
            res['log'] = (T.mean(axis=1) - C.mean(axis=1)) / np.log(2)
            res['log'][~gene_filter_]=0
            for ii in range(num_genes):
                if gene_filter_[ii]:
                    res['pval'][ii] , res['auc'][ii]  = run_stat(X_,C,T,y,ii)
        res = pd.DataFrame(res)
        g = d.var.reset_index()
        res['gene'] = g['index']
        res['pc'] = g['pc']
        res['qc'] = g['ercc'] | g['mt'] | g['rb']
        s = (res['pval']!=1) & (res['pc']) & (res['nzc']>1) & (res['nzt']>1) & (~res['qc'])
        res = res[s].reset_index()
        del res['index']
        a,p,a,a = multipletests(np.array(res['pval']), alpha=0.05, method='fdr_bh')
        res['pval_adj'] = p
        res = res.set_index('gene')
        return res

In [ ]:
folder = '../pb_results/'
data = {}
for tissue in tissues:
        print(tissue, end = ' ')
        data[tissue]= an.read_h5ad(folder + 'tissue_data_cpm_annotated_' + tissue + '.h5ad')

In [ ]:
data_ = {}
for tissue in tissues:
    print(tissue,end = ' ')
    data_[tissue] = {}
    d = data[tissue]
    a = d.obs
    for cell_type in a['cell_type_abbrev'].unique():
            s = (a['cell_type_abbrev']==cell_type)
            if s.sum()>0:
                data_[tissue][cell_type] = d[s,:]

In [ ]:
folder_ = '../pb_results/dge/'
for (co,tr) in [('Y','A'),('IA','HA'),('IY','HY')]:
  for tissue in data_:
    for cell_type in data_[tissue]:
        d = data_[tissue][cell_type]
        control = d.obs['condition']==co
        treatment = d.obs['condition']==tr
        if control.sum()>=10 and treatment.sum()>=10:
            res = dge(d,'condition',co,tr)
            code = co+tr
            res.to_pickle('dge_sc|' + code + '|' + tissue + '|' + cell_type + '.pkl')